# Austin Move Decision Making

In [ ]:
##Begin by importing libraries, setting up calls for later, etc

In [ ]:
#import libraries 
import pandas as pd 
import numpy as np 
import random 
import requests 
import json
import codecs
import os
import types
from urllib.request import urlopen
import plotly.express as px
import matplotlib.pylab as plt

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

! pip install folium==0.5.0
import folium 

print('Folium installed')
print('Libraries imported.')

In [ ]:
#add Foursquare information
CLIENT_ID = 'UOJC0SARIRT4OKOFRU5OHNLUKNS11BLSPUKF2MQSAVVJAL00' # your Foursquare ID
CLIENT_SECRET = '0RHBWZQXAAJ2BJGMYWOHMCSWEGCI05CQEBKUUVNBC2B3QRRN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
#add a location in central Austin to begin

address = '1100 Congress Ave, Austin, TX 78701'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

In [ ]:
#create a statment terrain map to show the terrain around austin
atx_map = folium.Map(location=[latitude, longitude], zoom_start = 12, tiles = 'Stamen Terrain')

atx_map

## Get Austin Data - Population, Crime Rates, Growth Rates

In [ ]:
#get crime information in Austin areas from https://data.austintexas.gov/Public-Safety/2018-Annual-Crime/pgvh-cpyq/data
#data is available in a json here - https://data.austintexas.gov/resource/pgvh-cpyq.json
#use it to create a dataframe with number of crimes by zip code

r = requests.get('https://data.austintexas.gov/resource/pgvh-cpyq.json')
j = r.json()
df_crimes1 = pd.DataFrame.from_dict(j)

df_crimes1.head()

In [ ]:
#clean the rows and columns to get a final dataframe with zip code and number of crimes per zip code

# clean up the dataset to remove unnecessary columns 
df_crimes1.drop(['go_primary_key_year_plus','go_highest_offense_desc','crime_type','go_report_date','go_location', 'go_x_coordinate', 'go_y_coordinate', 'clearance_status', 'clearance_date', 'go_district', 'council_district', 'go_census_tract'], axis=1, inplace=True)
df_crimes1.head()

In [ ]:
# add total column & rename the columns.  Call the zip codes column ZCTA5CE10 to match the json file later
df_crimes = df_crimes1.apply(pd.Series.value_counts).reset_index()
df_crimes.rename(columns={'index':'ZCTA5CE10', 'go_location_zip':'count'}, inplace=True)
df_crimes.head()

In [ ]:
#the above crimes are counts, not rates so now create a dataframe with population by zip code using data from here: http://zipatlas.com/us/tx/austin/zip-code-comparison/population-density.htm
#I created both a .csv and xls with this data but Watson Studio was not uploading them properly hence this manual step
population_data = [[78705, 26825], [78751, 14005], [78752, 18030], [78741, 40661], [78723, 30110], [78704, 43249], [78758, 42820], [78702, 22534], [78757, 21415], [78756, 7128], [78745, 53044], [78753, 44210], [78703, 19585], [78731, 24059], [78759, 40547], [78727, 22212], [78749, 28497], [78728, 17298], [78721, 10124], [78722, 6365], [78729, 24561], [78701, 3855], [78748, 25361], [78750, 23476], [78744, 33706], [78746, 26023], [78717, 8148], [78739, 8643], [78734, 12925], [78724, 15428], [78726, 6480], [78733, 8716], [78754, 5422], [78735, 9671], [78732, 3629], [78742, 625], [78730, 4885], [78738, 2840], [78747, 5079], [78736, 6897], [78737, 7100], [78725, 1836], [78719, 1765]]
               
df_pop = pd.DataFrame(population_data, columns = ['ZCTA5CE10', 'Population']) 
    
df_pop.head()

In [ ]:
#merge crimes with population to get crime rates rather than number of crimes:
df_pop['ZCTA5CE10']=df_pop['ZCTA5CE10'].astype(int)
df_crimes['ZCTA5CE10']=df_crimes['ZCTA5CE10'].astype(int)
df_crime_rate = df_pop.merge(df_crimes, left_on='ZCTA5CE10', right_on='ZCTA5CE10')

df_crime_rate['CrimeRate(%)'] = (df_crime_rate.fillna(0)['count'] / df_crime_rate.fillna(0)['Population']) *100

df_crime_rate.head()

In [ ]:
#create a choropleth crime map of Austin by zip code
map = folium.Map(location=[30.2672, -97.7431], zoom_start=12)

map.choropleth(geo_data="https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/tx_texas_zip_codes_geo.min.json",
             data=df_crime_rate, 
             columns=['ZCTA5CE10', 'count'], #ZCTA5CE10 is the name of the zip codes in the json file
             key_on='feature.properties.ZCTA5CE10', 
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Crimes Committed')
map

In [ ]:
#look at recent growth trends by zip code.  Found a chart at the following url with numbers, manually create it into a dataframe https://www.austintexas.gov/sites/default/files/files/Planning/Demographics/ZIPs_Sales_Price_and_Density.pdf
growth_data = [[78754, .28], [78719, .24], [78717, .22], [78722, .23], [78723, .22], [78725, .21], [78737, .18], [78702, .17], [78756, .15], [78726, .13], [78747, .14], [78704, .12], [78705, .11], [78757, .11], [78729, .10], [78751, .10], [78738, .9], [78735, .9], [78739, .9], [78724, .9], [78736, .8], [78703, .8], [78748, .7], [78758, .7], [78721, .7], [78752, .6], [78727, .6], [78728, .6], [78753, .6], [78744, .6],  [78741, .6], [78750, .5], [78745,.5], [78759, .4], [78734, .3], [78731, .3], [78749, .2], [78733, .2], [78746, .2], [78730, .1]]
               
df_growth = pd.DataFrame(growth_data, columns = ['ZCTA5CE10', 'GrowthRate']) 
    
df_growth.head()

In [ ]:
#merge crimes with growth to get a cooler chart:
df_growth['ZCTA5CE10']=df_growth['ZCTA5CE10'].astype(str)
df_crime_rate['ZCTA5CE10']=df_crime_rate['ZCTA5CE10'].astype(str)
df_crime_growth = df_growth.merge(df_crime_rate, left_on='ZCTA5CE10', right_on='ZCTA5CE10')

df_crime_growth.head()

In [ ]:
#create a chart showing growth vs crimes by zipcode
fig = px.bar(df_crime_growth, y='ZCTA5CE10', x='GrowthRate', color="CrimeRate(%)", orientation='h', color_continuous_scale='Bluered',
            labels={ "ZCTA5CE10": "Zip Code",
                     "GrowRate": "GrowthRate",
                     "CrimeRate(%)": "Crime Rate (%)"})
fig.update_layout(
   xaxis = dict (tickformat="%"),
   yaxis = dict (tickformat="##.##",
                 nticks=30))
    
fig.show()

## Get Austin Venue Information from Foursquare

In [ ]:
#pull in the dataset of Austin zip codes and their longitudes and latitudes.  I took this dataset and deleted all non-Austin entries https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/?refine.state=TX
#since I am using IBM Watson studio notebook this is how I have to import the data
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_4626cec52633482783cead9eb6658497 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='cKGIXmI8E5JX4WZiOy1ST1QU0bR9K4ijARilZcwXPDT_',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_4626cec52633482783cead9eb6658497.get_object(Bucket='applieddatasciencecapstone-donotdelete-pr-pyxclczerfkqqr',Key='us-zip-code-latitude-and-longitude.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_atx_long_lat = pd.read_csv(body)
df_atx_long_lat.head()

In [ ]:
#start getting venue info

LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, ', ', sep='', end='')
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['ZCTA5CE10', 
                  'INTPTLAT10', 
                  'INTPTLON10', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print('done')
    return(nearby_venues)

In [ ]:
df_venues = getNearbyVenues(names=df_atx_long_lat['Zip'],
                                   latitudes=df_atx_long_lat['Latitude'],
                                   longitudes=df_atx_long_lat['Longitude'])
print(df_venues.shape)

In [ ]:
#create a table with venue categories across the top and zip codes down the side and get counts:
df_venues['COUNTER'] =1       #initially, set that counter to 1.
group_data = df_venues.groupby(['Venue Category','ZCTA5CE10'])['COUNTER'].sum()
df_venue_by_zip = df_venues.groupby(['Venue Category','ZCTA5CE10'])['COUNTER'].sum().reset_index()
#df_venue_by_zip.head()

df_venues_pivot = pd.pivot_table(df_venue_by_zip, values = 'COUNTER', index=['ZCTA5CE10'], columns = 'Venue Category').reset_index()
df_venues_pivot.head()

In [ ]:
#get a list of all of the venue categories to determine which are interesting
pd.set_option('display.max_rows', None)
df_ven_cats = df_venues.groupby(['Venue Category']).size()
print(df_ven_cats)

In [ ]:
#drop columns our family is not interested in
df_venues_pivot.drop(['Adult Boutique', 'Arcade', 'Art Gallery', 'Art Museum', "Arts & Crafts Store", 'Assisted Living', 'Automotive Shop', 'BBQ Joint', 'Bagel Shop', 'Bakery', 'Bank', 'Basketball Court', 'Bed & Breakfast', 'Beer Bar', 'Beer Garden', 'Beer Store', 'Big Box Store', 'Boutique', 'Brewery', 'Bus Station', 'Bus Stop', 'Business Service', 'Butcher', 'Cajun / Creole Restaurant', 'Capitol Building', 'Car Wash', 'Casino', 'Chinese Restaurant', 'Chiropractor', 'Clothing Store', 'College Bookstore', 'College Gym', 'College Quad', 'Comedy Club', 'Comfort Food Restaurant', 'Concert Hall', 'Construction & Landscaping', 'Convenience Store', 'Convention Center', 'Cosmetics Shop', 'Cupcake Shop', 'Cycle Studio', 'Dance Studio', 'Deli / Bodega', 'Department Store', 'Dessert Shop', 'Diner', 'Discount Store', 'Dive Bar', "Doctor's Office", 'Dog Run', 'Donut Shop', 'Dry Cleaner', 'Electronics Store', 'Event Service', 'Fabric Shop', 'Farm', 'Fast Food Restaurant', 'Field', 'Financial or Legal Service', 'Flea Market', 'Flower Shop', 'Food', 'Food & Drink Shop', 'Food Court', 'Food Stand', 'Football Stadium', 'Fountain', 'French Restaurant', 'Fried Chicken Joint', 'Frozen Yogurt Shop', 'Furniture / Home Store', 'Garden', 'Gas Station', 'Gastropub', 'Gay Bar', 'General Entertainment', 'Gift Shop', 'Golf Driving Range', 'Gourmet Shop', 'Greek Restaurant', 'Health & Beauty Service', 'Historic Site', 'History Museum', 'Hobby Shop', 'Home Service', 'Hostel', 'Hot Dog Joint', 'Hotel', 'Hotel Bar', 'Ice Cream Shop', 'Indian Restaurant', 'Intersection', 'Italian Restaurant', 'Juice Bar', 'Karaoke Bar', 'Kitchen Supply Store', 'Lake', 'Lawyer', 'Lighthouse', 'Lingerie Store', 'Liquor Store', 'Lounge', 'Market', 'Martial Arts School', 'Massage Studio', "Men's Store", 'Miscellaneous Shop', 'Mobile Phone Shop', 'Monument / Landmark', 'Motel', 'Motorcycle Shop', 'Motorsports Shop', 'Mountain', 'Movie Theater', 'Museum', 'Music Store', 'Music Venue', 'Nail Salon', 'New American Restaurant', 'Nightclub', 'Optical Shop', 'Outdoor Sculpture', 'Pastry Shop', 'Performing Arts Venue', 'Pharmacy', 'Pizza Place', 'Playground', 'Pool', 'Public Art', 'Ramen Restaurant', 'Record Shop', 'Recording Studio', 'Rental Car Location', 'Rental Service', 'Restaurant', 'Rock Club', 'Russian Restaurant', 'Salad Place', 'Salon / Barbershop', 'Sandwich Place', 'Sculpture Garden', 'Seafood Restaurant', 'Shipping Store', 'Shoe Store', 'Shop & Service', 'Shopping Mall', 'Skating Rink', 'Ski Shop', 'Smoke Shop', 'Smoothie Shop', 'Snack Place', 'Soccer Field', 'Soccer Stadium', 'Soup Place', 'South American Restaurant', 'Southern / Soul Food Restaurant', 'Souvenir Shop', 'Spa', 'Speakeasy', 'Sporting Goods Shop', 'Stables', 'Steakhouse', 'Storage Facility', 'Student Center', 'Supplement Shop', 'Tailor Shop', 'Tanning Salon', 'Tapas Restaurant', 'Tennis Court', 'Theater', 'Thrift / Vintage Store', 'Toy / Game Store', 'Trade School', 'Tree', 'Vegetarian / Vegan Restaurant', 'Video Game Store', 'Video Store', 'Vietnamese Restaurant', 'Volleyball Court', 'Weight Loss Center', 'Wine Shop', 'Wings Joint', "Women's Store", 'Yoga Studio'], axis=1, inplace=True)
df_venues_pivot.head()

In [ ]:
#create a cleaner final data table
#assume listings are mutually exclusive and create higher level categories:
df_venues_pivot['Restaurants'] =  (df_venues_pivot.fillna(0)['American Restaurant'] + df_venues_pivot.fillna(0)['Asian Restaurant'] + df_venues_pivot.fillna(0)['Breakfast Spot'] + df_venues_pivot.fillna(0)['Burger Joint'] + 
                                   df_venues_pivot.fillna(0)['Burrito Place'] + df_venues_pivot.fillna(0)['Taco Place'] + df_venues_pivot.fillna(0)['Tex-Mex Restaurant'] + df_venues_pivot.fillna(0)['Thai Restaurant'] + 
                                   df_venues_pivot.fillna(0)['Café'] + df_venues_pivot.fillna(0)['Falafel Restaurant'] + df_venues_pivot.fillna(0)['Food Truck'] + df_venues_pivot.fillna(0)['Mexican Restaurant'] +
                                   df_venues_pivot.fillna(0)['Japanese Restaurant'] + df_venues_pivot.fillna(0)['Latin American Restaurant'] + df_venues_pivot.fillna(0)['Mediterranean Restaurant'] +
                                   df_venues_pivot.fillna(0)['Middle Eastern Restaurant'] + df_venues_pivot.fillna(0)['Moroccan Restaurant'] + df_venues_pivot.fillna(0)['Kebab Restaurant'] + 
                                   df_venues_pivot.fillna(0)['Korean Restaurant'] +  df_venues_pivot.fillna(0)['Sushi Restaurant']) 

df_venues_pivot['Bars'] = (df_venues_pivot.fillna(0)['Bar'] + df_venues_pivot.fillna(0)['Pub'] + df_venues_pivot.fillna(0)['Whisky Bar'] + df_venues_pivot.fillna(0)['Cocktail Bar'] +
                           df_venues_pivot.fillna(0)['Piano Bar']) 

df_venues_pivot['Essentials'] = (df_venues_pivot.fillna(0)['Grocery Store'] + df_venues_pivot.fillna(0)['Pet Store'] + df_venues_pivot.fillna(0)['Health Food Store'])

df_venues_pivot['Gyms'] =  (df_venues_pivot.fillna(0)['Gym'] + df_venues_pivot.fillna(0)['Gym / Fitness Center'] + df_venues_pivot.fillna(0)['Recreation Center'])

df_venues_pivot['Entertainment'] =  (df_venues_pivot.fillna(0)['Bookstore'] + df_venues_pivot.fillna(0)['Bubble Tea Shop'] + df_venues_pivot.fillna(0)['Coffee Shop'] + 
                                     df_venues_pivot.fillna(0)['Hookah Bar'] + df_venues_pivot.fillna(0)['Jazz Club'] + df_venues_pivot.fillna(0)['Pool Hall'])

#clean the extra columns:
df_venues_pivot.drop(['American Restaurant', 'Asian Restaurant', 'Breakfast Spot', 'Burger Joint', 'Burrito Place', 'Taco Place', 'Tex-Mex Restaurant', 'Thai Restaurant', 'Café', 'Falafel Restaurant', 'Food Truck'], axis=1, inplace=True)
df_venues_pivot.drop(['Mexican Restaurant', 'Japanese Restaurant', 'Latin American Restaurant', 'Mediterranean Restaurant', 'Middle Eastern Restaurant', 'Moroccan Restaurant', 'Kebab Restaurant', 'Korean Restaurant', 'Sushi Restaurant'], axis=1, inplace=True)

df_venues_pivot.drop(['Bar', 'Pub', 'Whisky Bar', 'Piano Bar', 'Cocktail Bar'], axis=1, inplace=True)

df_venues_pivot.drop(['Grocery Store', 'Pet Store', 'Health Food Store'], axis=1, inplace=True)

df_venues_pivot.drop(['Gym', 'Gym / Fitness Center', 'Recreation Center'], axis=1, inplace=True)

df_venues_pivot.drop(['Bookstore', 'Bubble Tea Shop', 'Coffee Shop', 'Hookah Bar', 'Jazz Club', 'Pool Hall'], axis=1, inplace=True)

#check the output:
df_venues_pivot.head()

In [ ]:
#clean the extraneous zip codes out of the dataframe
zips = [[78754], [78719], [78717], [78722], [78723], [78725], [78737], [78702], [78756], [78726], [78747], [78704], [78705], [78757], [78729], [78751], [78738], [78735], [78739], [78724], [78736], [78703], [78748], [78758], [78721], [78752], [78727], [78728], [78753], [78744], [78741], [78750], [78745], [78759], [78734], [78731], [78749], [78733], [78746], [78730]]
df_zips = pd.DataFrame(zips, columns = ['ZCTA5CE10']) 

df_venues_pivot2 = df_zips.merge(df_venues_pivot, on='ZCTA5CE10',how='inner', indicator=True)
df_venues_pivot2.drop(['_merge'], axis=1, inplace=True)

df_venues_pivot2.head()

In [ ]:
#create a stacked bar graph of venue information

ax1 = df_venues_pivot2.groupby(['ZCTA5CE10']).sum().fillna(0)

ax = ax1.plot(kind='bar', stacked=True, figsize=(50,25), fontsize=20, legend=20)

ax.legend(fontsize=20)

In [ ]:
#create a cleaner version with total Venues for use later
df_ttl_venue = df_venues_pivot2.copy()
df_ttl_venue['All Venues'] = (df_ttl_venue.fillna(0)['Athletics & Sports'] + df_ttl_venue.fillna(0)['Park'] + df_ttl_venue.fillna(0)['Scenic Lookout'] + df_ttl_venue.fillna(0)['Trail'] +
                              df_ttl_venue.fillna(0)['Restaurants'] + df_ttl_venue.fillna(0)['Bars'] + df_ttl_venue.fillna(0)['Essentials'] +
                              df_ttl_venue.fillna(0)['Gyms'] + df_ttl_venue.fillna(0)['Entertainment'])
df_ttl_venue.drop(['Athletics & Sports', 'Park', 'Scenic Lookout', 'Trail', 'Restaurants', 'Bars', 'Essentials', 'Gyms', 'Entertainment'], axis=1, inplace=True)

df_ttl_venue.head()

## Get Austin Rent Information

In [ ]:
#now create a dataframe with rent by zip code using data from here: https://www.homecity.com/blog/the-cheapest-and-most-expensive-austin-zip-codes/
rent_data = [[78724, 612], [78721, 612], [78664, 807], [78660, 807], [78728, 807], [78717, 807], [78726, 807], [78750, 807], [78757, 807], [78752, 807], [78723, 807], [78741, 807], [78744, 807], [78747, 807], [78748, 807], [78681, 1006], [78729, 1006], [78727, 1006], [78759, 1006], [78737, 1006], [78756, 1006], [78751, 1006], [78722, 1006], [78732, 1006], [78730, 1006], [78736, 1006], [78749, 1006], [78745, 1006], [78613, 1236], [78735, 1236], [78746, 1236], [78703, 1236], [78705, 1236], [78702, 1236], [78704, 1236], [78737, 1686]]
               
df_rent = pd.DataFrame(rent_data, columns = ['ZCTA5CE10', 'Avg Rent']) 
    
df_rent.head()

In [ ]:
#merge the datasets together
df_rent['ZCTA5CE10']=df_rent['ZCTA5CE10'].astype(str)
df_crime_rate['ZCTA5CE10']=df_crime_rate['ZCTA5CE10'].astype(str)

df_rent_crime = df_rent.merge(df_crime_rate, left_on='ZCTA5CE10', right_on='ZCTA5CE10')

df_ttl_venue['ZCTA5CE10']=df_ttl_venue['ZCTA5CE10'].astype(str).reset_index(drop=True)
df_rent_crime['ZCTA5CE10']=df_rent_crime['ZCTA5CE10'].astype(str).reset_index(drop=True)

df_all = df_rent_crime.merge(df_ttl_venue, left_on='ZCTA5CE10', right_on='ZCTA5CE10')
df_all.head()

In [ ]:
#Compare rent to total number of venues to crime rate - couldn't get this to color by zip code so have to compare back to the combined dataset once you find bubbles of interest

#add a column to make bubbles easier to see:
df_all['Venue_bigger'] = (df_all.fillna(0)['All Venues'] ) *20

plt.scatter('Avg Rent', 'CrimeRate(%)',
             s='Venue_bigger',
             alpha=.5, data=df_all)
plt.xlabel("Avg Rent", size=16)
plt.ylabel("Crime Rate (%)", size=16)
plt.title("Rent vs Crimes vs Number of Venues by Zip Code", size=18)

In [ ]:
#get a final dataset with all relevant data by zip code
df_all['ZCTA5CE10']=df_all['ZCTA5CE10'].astype(str)
df_venues_pivot2['ZCTA5CE10']=df_venues_pivot2['ZCTA5CE10'].astype(str)

df_all2 = df_all.merge(df_venues_pivot2, left_on='ZCTA5CE10', right_on='ZCTA5CE10')

df_all2['ZCTA5CE10']=df_ttl_venue['ZCTA5CE10'].astype(str)
df_growth['ZCTA5CE10']=df_growth['ZCTA5CE10'].astype(str)

df_all3 = df_all2.merge(df_growth, left_on='ZCTA5CE10', right_on='ZCTA5CE10')

df_all3.drop(['Venue_bigger'], axis=1, inplace=True)
df_all3.rename(columns={'count':'Total Crimes', 'ZCTA5CE10':'Zip Code'}, inplace=True)

df_all3.head()